
# Basic Brain Decoding on EEG Data

This tutorial shows you how to train and test deep learning models with
Braindecode in a classical EEG setting: you have trials of data with
labels (e.g., Right Hand, Left Hand, etc.).
   :depth: 2


## Loading and preparing the data




In [1]:
from braindecode.datasets import MOABBDataset
import os
# from moabb.datasets import download
# from mne import get_config, set_config
from pathlib import Path

c:\Users\mengz\anaconda3\envs\hyperBCI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\mengz\anaconda3\envs\hyperBCI\Lib\site-packages\moabb\pipelines\__init__

### Loading the dataset




First, we load the data. In this tutorial, we load the BCI Competition
IV 2a data [1]_ using braindecode's wrapper to load via
[MOABB library](https://github.com/NeuroTechX/moabb)_ [2]_.

<div class="alert alert-info"><h4>Note</h4><p>To load your own datasets either via mne or from
   preprocessed X/y numpy arrays, see [MNE Dataset
   Tutorial](./plot_mne_dataset_example.html)_ and [Numpy Dataset
   Tutorial](./plot_custom_dataset_example.html)_.</p></div>




In [2]:
# # assuming current working directory is Hypernet/notebooks
# repo_path = os.path.dirname(os.getcwd())
# # set dataset location environment variable
dataset_name = "BNCI2014_001"
# # dataset_name = 'BNCI2014-001'
# dir_data = os.path.join(repo_path, 'data')
# cache_config = {'path': dir_data}
# set_config('MNE_DATASETS_BNCI2014-001_PATH', dir_data)
# set_config('MNE_DATASETS_BNCI2014_001_PATH', dir_data)
# # os.environ[f'MNE_DATASETS_{dataset_name}_PATH'] = os.path.join(repo_path, 'data')

subject_id = 2
dataset = MOABBDataset(dataset_name=dataset_name, subject_ids=[subject_id])
# dataset = MOABBDataset(dataset_name=dataset_name, subject_ids=[subject_id], dataset_kwargs=None, dataset_load_kwargs=cache_config)
# dataset = MOABBDataset(dataset_name, [subject_id,], None, cache_config)

In [4]:
# get_config(f"MNE_DATASETS_BNCI2014-001_PATH")

In [5]:
# # download.get_dataset_path(dataset_name, None)
# test = Path(get_config("MNE_DATA", get_config("MNE_DATA"))).expanduser()
# test.exists()

### Preprocessing




Now we apply preprocessing like bandpass filtering to our dataset. You
can either apply functions provided by
[mne.Raw](https://mne.tools/stable/generated/mne.io.Raw.html)_ or
[mne.Epochs](https://mne.tools/0.11/generated/mne.Epochs.html#mne.Epochs)_
or apply your own functions, either to the MNE object or the underlying
numpy array.

<div class="alert alert-info"><h4>Note</h4><p>Generally, braindecode prepocessing is directly applied to the loaded
   data, and not applied on-the-fly as transformations, such as in
   PyTorch-libraries like
   [torchvision](https://pytorch.org/docs/stable/torchvision/index.html)_.</p></div>




In [3]:
from numpy import multiply

from braindecode.preprocessing import (Preprocessor,
                                       exponential_moving_standardize,
                                       preprocess)

low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000
# Factor to convert from V to uV
factor = 1e6

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda data: multiply(data, factor)),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
preprocess(dataset, preprocessors, n_jobs=-1)

c:\Users\mengz\anaconda3\envs\hyperBCI\Lib\site-packages\braindecode\preprocessing\preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


### Extracting Compute Windows




Now we extract compute windows from the signals, these will be the inputs
to the deep networks during training. In the case of trialwise
decoding, we just have to decide if we want to include some part
before and/or after the trial. For our work with this dataset,
it was often beneficial to also include the 500 ms before the trial.




In [4]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


### Splitting the dataset into training and validation sets




We can easily split the dataset using additional info stored in the
description attribute, in this case ``session`` column. We select
``T`` for training and ``test`` for validation.




In [4]:
# from braindecode.datasets import BaseConcatDataset
# from braindecode.datasets.base import EEGWindowsDataset

In [5]:
splitted_lst_by_run = list(windows_dataset.split('run').values())
# valid_set_size = 1
# train_set = BaseConcatDataset(splitted_lst_by_run[:-valid_set_size])
# valid_set = BaseConcatDataset(splitted_lst_by_run[-valid_set_size:])
splitted_dict_by_ses = windows_dataset.split('session')
train_set = splitted_dict_by_ses.get('0train')
valid_set = splitted_dict_by_ses.get('1test')

In [9]:
# train_set.get_metadata()

## Creating a model




Now we create the deep learning model! Braindecode comes with some
predefined convolutional neural network architectures for raw
time-domain EEG. Here, we use the shallow ConvNet model from [3]_. These models are
pure [PyTorch](https://pytorch.org)_ deep learning models, therefore
to use your own model, it just has to be a normal PyTorch
[nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)_.




In [6]:
import torch
from braindecode.models import EEGConformer, ShallowFBCSPNet, EEGResNet, EEGNetv4
from braindecode.util import set_random_seeds

In [17]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# print(n_chans)
# print(input_window_samples)
# model = EEGNetv4(
#     n_chans,
#     n_classes,
#     final_conv_length='auto',
#     input_window_samples=input_window_samples
# )

# model = EEGConformer(
#     n_outputs=n_classes,
#     n_chans=n_chans,
#     input_window_samples=input_window_samples,
#     sfreq=sfreq,
#     # return_features=False
# )

# Display torchinfo table describing the model
# print(model)

# Send model to GPU
if cuda:
    model = model.cuda()

c:\Users\mengz\anaconda3\envs\hyperBCI\Lib\site-packages\braindecode\models\base.py:23: UserWarning: ShallowFBCSPNet: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
c:\Users\mengz\anaconda3\envs\hyperBCI\Lib\site-packages\braindecode\models\base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [16]:
# for name, param in model.named_parameters():
#     print(name)
#     print(param.requires_grad)

## Model Training




Now we will train the network! ``EEGClassifier`` is a Braindecode object
responsible for managing the training of neural networks. It inherits
from skorch [NeuralNetClassifier](https://skorch.readthedocs.io/en/stable/classifier.html#)_,
so the training logic is the same as in [Skorch](https://skorch.readthedocs.io/en/stable/)_.




<div class="alert alert-info"><h4>Note</h4><p>In this tutorial, we use some default parameters that we
   have found to work well for motor decoding, however we strongly
   encourage you to perform your own hyperparameter optimization using
   cross validation on your training data.</p></div>




In [18]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier

# We found these values to be good for the shallow network:
# lr = 0.0625 * 0.01
# weight_decay = 0

# For deep4 they should be:
lr = 1 * 0.01
weight_decay = 0.5 * 0.001

batch_size = 64
n_epochs = 40

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
    classes=classes,
)
clf.initialize()

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 22, 1125]             [1, 4]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 22, 1125]             [1, 22, 1125, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 22, 1125, 1]          [1, 1, 1125, 22]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1125, 22]          [1, 40, 1101, 1]          36,240                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1101, 1]          [1, 40, 1101, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

In [99]:
for name, _ in clf.module.named_parameters():
    print(name)

conv_time_spat.conv_time.weight
conv_time_spat.conv_time.bias
conv_time_spat.conv_spat.weight
bnorm.weight
bnorm.bias
final_layer.conv_classifier.weight
final_layer.conv_classifier.bias


In [95]:
## Load a pretrained model then try to freeze certain layers
dir_repo = os.path.dirname(os.getcwd())
dir_pretrain = os.path.join(dir_repo, 'pretrained')
temp_exp_name = 'baseline_2_3_pretrain'
holdout_subj_id = '1'
clf.load_params(f_params=os.path.join(dir_pretrain, f'{temp_exp_name}_without_subj_{holdout_subj_id}_model.pkl'), 
                f_optimizer=os.path.join(dir_pretrain, f'{temp_exp_name}_without_subj_{holdout_subj_id}_opt.pkl'), 
                f_history=os.path.join(dir_pretrain, f'{temp_exp_name}_without_subj_{holdout_subj_id}_history.json'))

In [96]:
def freeze_param(module_obj, parameter_name):
    param = module_obj
    for t in parameter_name.split('.'):
        param = getattr(param, t)
    param.requires_grad = False

param_name = 'final_layer.conv_classifier.weight'
freeze_param(clf.module, param_name)

In [97]:
print(clf.module.conv_time_spat.conv_time.weight.requires_grad)
# clf.module.conv_time_spat.conv_time.weight
# clf.module.final_layer.conv_classifier.weight

False


In [86]:
# Model training for the specified number of epochs. `y` is None as it is
# already supplied in the dataset.
n_epochs = 5
_ = clf.fit(train_set, y=None, epochs=n_epochs)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  ------
      1            0.2778        1.6668       0.2535            0.2535        3.2631  0.0100  0.7228
      2            0.3333        1.5113       0.2396            0.2396        2.2828  0.0100  0.9999
      3            0.3785        1.4899       0.2639            0.2639        1.9896  0.0099  0.9522
      4            0.4271        1.3945       0.2500            0.2500        1.8145  0.0099  0.9507
      5            0.4618        1.3719       0.2708            0.2708        1.7020  0.0097  0.9500


In [87]:
# print(clf.module.final_layer.conv_classifier.weight.requires_grad)
# clf.module.final_layer.conv_classifier.weight

print(clf.module.conv_time_spat.conv_time.weight.requires_grad)
clf.module.conv_time_spat.conv_time.weight

False


Parameter containing:
tensor([[[[-2.4285e-03],
          [-8.7658e-02],
          [ 1.3998e-02],
          [-2.0006e-03],
          [ 1.4557e-02],
          [-4.7127e-02],
          [-6.8500e-02],
          [-5.9299e-02],
          [ 6.1342e-02],
          [ 6.7051e-02],
          [-2.3713e-02],
          [ 1.6661e-02],
          [-4.1164e-03],
          [ 3.1189e-02],
          [-2.8836e-02],
          [ 8.3128e-02],
          [-5.1969e-02],
          [-5.5538e-02],
          [-8.8960e-02],
          [-5.9401e-02],
          [ 2.4759e-02],
          [ 6.7848e-02],
          [-3.5644e-02],
          [ 6.5016e-02],
          [-2.7443e-03]]],


        [[[ 5.2974e-02],
          [-3.5225e-02],
          [ 4.3739e-02],
          [ 9.5614e-02],
          [ 3.2238e-02],
          [-6.2332e-02],
          [-7.8644e-02],
          [-5.3223e-02],
          [-9.0229e-02],
          [-5.7769e-02],
          [-5.0814e-02],
          [ 7.1697e-02],
          [ 4.9970e-02],
          [ 7.7705e-04],

In [48]:
clf.module.bnorm.weight

Parameter containing:
tensor([1.1111, 1.0469, 1.0447, 1.0566, 1.0859, 1.0797, 1.0563, 1.0683, 1.0467,
        1.0627, 1.0672, 1.0532, 1.1149, 1.0771, 1.0817, 1.0735, 1.0715, 1.0716,
        1.0753, 1.0318, 1.0635, 1.0861, 1.0747, 1.0976, 1.0744, 1.0534, 1.0841,
        1.0573, 1.0456, 1.0773, 1.0866, 1.0605, 1.0609, 1.0544, 1.0980, 1.0626,
        1.0527, 1.0831, 1.0828, 1.0920], requires_grad=True)

## Plotting Results




Now we use the history stored by Skorch throughout training to plot
accuracy and loss curves.




In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.lines import Line2D

# Extract loss and accuracy values for plotting from history object
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns=results_columns,
                  index=clf.history[:, 'epoch'])

# get percent of misclass for better visual comparison to loss
df = df.assign(train_misclass=100 - 100 * df.train_accuracy,
               valid_misclass=100 - 100 * df.valid_accuracy)

fig, ax1 = plt.subplots(figsize=(8, 3))
df.loc[:, ['train_loss', 'valid_loss']].plot(
    ax=ax1, style=['-', ':'], marker='o', color='tab:blue', legend=False, fontsize=14)

ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.set_ylabel("Loss", color='tab:blue', fontsize=14)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

df.loc[:, ['train_misclass', 'valid_misclass']].plot(
    ax=ax2, style=['-', ':'], marker='o', color='tab:red', legend=False)
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=14)
ax2.set_ylabel("Misclassification Rate [%]", color='tab:red', fontsize=14)
ax2.set_ylim(ax2.get_ylim()[0], 85)  # make some room for legend
ax1.set_xlabel("Epoch", fontsize=14)

# where some data has already been plotted to ax
handles = []
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle='-', label='Train'))
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle=':', label='Valid'))
plt.legend(handles, [h.get_label() for h in handles], fontsize=14)
plt.tight_layout()

## Plotting a  Confusion Matrix




Here we generate a confusion matrix as in [3]_.




In [ ]:
from sklearn.metrics import confusion_matrix

from braindecode.visualization import plot_confusion_matrix

# generate confusion matrices
# get the targets
y_true = valid_set.get_metadata().target
y_pred = clf.predict(valid_set)

# generating confusion matrix
confusion_mat = confusion_matrix(y_true, y_pred)

# add class labels
# label_dict is class_name : str -> i_class : int
label_dict = windows_dataset.datasets[0].window_kwargs[0][1]['mapping']
# sort the labels by values (values are integer class labels)
labels = [k for k, v in sorted(label_dict.items(), key=lambda kv: kv[1])]

# plot the basic conf. matrix
plot_confusion_matrix(confusion_mat, class_names=labels)

## References

.. [1] Tangermann, M., Müller, K.R., Aertsen, A., Birbaumer, N., Braun, C.,
       Brunner, C., Leeb, R., Mehring, C., Miller, K.J., Mueller-Putz, G.
       and Nolte, G., 2012. Review of the BCI competition IV.
       Frontiers in neuroscience, 6, p.55.

.. [2] Jayaram, Vinay, and Alexandre Barachant.
       "MOABB: trustworthy algorithm benchmarking for BCIs."
       Journal of neural engineering 15.6 (2018): 066011.

.. [3] Schirrmeister, R.T., Springenberg, J.T., Fiederer, L.D.J., Glasstetter, M.,
       Eggensperger, K., Tangermann, M., Hutter, F., Burgard, W. and Ball, T. (2017),
       Deep learning with convolutional neural networks for EEG decoding and visualization.
       Hum. Brain Mapping, 38: 5391-5420. https://doi.org/10.1002/hbm.23730.

